# 4998 Applied Deep Learning Project
## Detect Cancer Metastases on Pathology Images

Jing Qian (jq2282)

# STEP 1. First glance of data and generate training images

## Step 1 is processed in GenerateTrain.ipynb.



In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
total_path = '/content/drive/My Drive/project-adl'

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray

In [0]:
input_size = 299 # in corresponding level
tumor_check_size = 128 # in level 0 with the highest resolution

# STEP 2. MODELING

* Although transfer learning may be less effective than training a model from scratch , it is the best place to start.
* Choose a model previously trained on Imagenet. Use the techniques in Chapter 5 of Francois’s book to try transfer learning (add a single Dense layer on top of that model), and train it on your own data.
* Write a script that takes your trained model and a testing image, and outputs a heat map showing the cancerous regions.
* Design an evaluation metric, write a script to report your results

加快速度，不看背景，只看tissue。比如对色彩做个threshold

选一个特别放大的开始做，比如800\*600 pixel图片
annotation. 选择哪部分有cancer

image segmentation, 


In [0]:
TARGET_SHAPE = 299
BATCH_SIZE = 32

## 2.0 Get Training Images

In [0]:
train_dir = 'drive/My Drive/project-adl/train_level4'

In [0]:
nclass=200

## 2.1 Transfer learning

In [20]:
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

conv_base = InceptionV3(weights='imagenet',include_top=False, input_shape=(299,299,3))
#conv_base.summary()
datagen = ImageDataGenerator(rescale=1./255)

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 8, 8, 2048)) # get from conv_base.summary()
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(TARGET_SHAPE, TARGET_SHAPE),
        batch_size=BATCH_SIZE,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)       
        features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = features_batch
        labels[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = labels_batch
        i += 1
        if i * BATCH_SIZE >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, nclass*2)
print(train_labels)

FLATTENED_SHAPE = 8 * 8 * 2048


Found 400 images belonging to 2 classes.
[1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1.
 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1.
 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1.
 0. 1. 1. 

In [23]:
train_features = np.reshape(train_features, (nclass*2, FLATTENED_SHAPE))
EPOCHS = 50
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=FLATTENED_SHAPE))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history3 = model.fit(train_features, train_labels,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split = 0.2)

Train on 320 samples, validate on 80 samples
Epoch 1/50
320/320 [==============================] - 1s 4ms/sample - loss: 2.9683 - acc: 0.7406 - val_loss: 1.7289 - val_acc: 0.8875
Epoch 2/50
320/320 [==============================] - 1s 2ms/sample - loss: 2.4292 - acc: 0.8375 - val_loss: 1.3472 - val_acc: 0.9125
Epoch 3/50
320/320 [==============================] - 1s 2ms/sample - loss: 2.6834 - acc: 0.8281 - val_loss: 1.1755 - val_acc: 0.9250
Epoch 4/50
320/320 [==============================] - 1s 2ms/sample - loss: 2.3100 - acc: 0.8531 - val_loss: 1.5942 - val_acc: 0.9000
Epoch 5/50
320/320 [==============================] - 1s 2ms/sample - loss: 1.9793 - acc: 0.8719 - val_loss: 1.0008 - val_acc: 0.9375
Epoch 6/50
320/320 [==============================] - 1s 2ms/sample - loss: 2.6346 - acc: 0.8344 - val_loss: 1.1300 - val_acc: 0.9250
Epoch 7/50
320/320 [==============================] - 1s 2ms/sample - loss: 2.4059 - acc: 0.8469 - val_loss: 1.7164 - val_acc: 0.8875
Epoch 8/50
320/32

## 2.2 Train from scratch

In [0]:
conv_base_2 = InceptionV3(weights=None,include_top=False, input_shape=(299,299,3))
#conv_base_2.summary()
datagen = ImageDataGenerator(rescale=1./255)

def extract_features_2(directory, sample_count):
    features = np.zeros(shape=(sample_count, 8, 8, 2048)) # get from conv_base.summary()
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(TARGET_SHAPE, TARGET_SHAPE),
        batch_size=BATCH_SIZE,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base_2.predict(inputs_batch)       
        features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = features_batch
        labels[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = labels_batch
        i += 1
        if i * BATCH_SIZE >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features_2(train_dir, nclass*2)
print(train_labels)

FLATTENED_SHAPE = 8 * 8 * 2048

In [0]:
train_features = np.reshape(train_features, (nclass*2, FLATTENED_SHAPE))
EPOCHS = 50
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=FLATTENED_SHAPE))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history3 = model.fit(train_features, train_labels,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

## 1.4 Generate test patches and corresponding labels

### Start with one slide

In [1]:
# Install the OpenSlide C library and Python bindings
!apt-get install openslide-tools
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [0]:
# As mentioned in class, we can improve efficiency by ignoring non-tissue areas 
# of the slide. We'll find these by looking for all gray regions.
def find_tissue_pixels(image, intensity=0.8):
    im_gray = rgb2gray(image)
    assert im_gray.shape == (image.shape[0], image.shape[1])
    indices = np.where(im_gray <= intensity)
    return zip(indices[0], indices[1])

# This function has nothing to do with tumor mask, but shows tissue region
def apply_mask(im, mask, color=(255,0,0)):
    masked = np.copy(im)
    for x,y in mask: masked[x][y] = color
    return masked

In [0]:
# See https://openslide.org/api/python/#openslide.OpenSlide.read_region
# Note: x,y coords are with respect to level 0.
# There is an example below of working with coordinates
# with respect to a higher zoom level.

# Read a region from the slide
# Return a numpy RBG array
def read_slide(slide, x, y, level, width, height, as_float=False):
    #print('1')
    im = slide.read_region((x,y), level, (width, height))
    #print('2')
    im = im.convert('RGB') # drop the alpha channel
    #print('3')
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im